In [1]:
import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [2]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations

In [3]:
import imgaug as ia
from imgaug import augmenters as iaa
from files import *
from pascal_voc_writer import Writer
from tqdm import tqdm_notebook

In [4]:
# 사물 rotation (bounding box도 그에 맞게 rotation)
def rotation(input_dir, output_dir):
    ia.seed(1)
    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Sequential([
            iaa.Affine(
                translate_px={"x": (0, 5), "y": (-15, -10)}, # 사진 중심 위치 조정하는 것. # (0,5)는 x축이 0에서 5사이로 랜덤으로 하게
                # ex) x값이 음수이고 절댓값 클수록 왼쪽으로
                scale =(0.65, 0.7), # 크기. 0.65배~0.7배 사이 랜덤으로 크기 축소함.
                rotate= (-90, 90) # 회전하는 각도.
                # 시행착오 겪으며 설정한 지금의 scale, translate_px 값이 제일 괜찮은 듯 합니다.
            )
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [5]:
# blur 처리
def blur(input_dir, output_dir):
    ia.seed(1)
    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.GaussianBlur(sigma=(2.0, 3.0)) # 가우시안 블러 처리

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [6]:
# noise 처리
def noise(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.AdditiveGaussianNoise(scale=(0, 35)) # 노이즈 중에 가우시안 노이즈 채택 (scale 최댓값 0.2*255)
        # 모델 학습에 좋은 노이즈. 적용해봤을 때도 크게 과하지 않는 정규분포를 띠는 노이즈

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [7]:
# bright 처리
def bright(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Multiply((1.25, 1.5)) # 1.25~1.5배 사이에서 랜덤으로 밝아짐 (0.5~1.5 값만 설정 가능)
        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [8]:
# 박스 랜덤 위치에 생성
def box(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Cutout(nb_iterations=(1, 5), size=0.15, squared=False) # 박스1~5개 랜덤 생성

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [12]:
# 모션 블러 (움직이다가 찍은 것과 같은 효과를 줌)
def motionblur(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.MotionBlur(k=15)
        # 15x15 픽셀만큼 이동한 효과를 준다.
        # angle 옵션은 따로 설정하지 않아 여러 각도에서 랜덤하게 움직인 것처럼 해준다.

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [18]:
# 채도 조정(채도를 높이거나 낮춤)
def color(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.pillike.EnhanceColor(0.65, 3)
        # 1일 경우 채도를 그대로 둔다. 이를 기준으로
        # 랜덤하게 채도를 높이거나 낮춘다.

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [23]:
# 픽셀화
def pixel(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        points_sampler = iaa.RegularGridPointsSampler(n_cols=20, n_rows=40)
        # column 20개, row 40개로 구성된 픽셀 이미지를 구성한다
        seq = iaa.Voronoi(points_sampler)
        
        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [30]:
# 선명도 랜덤 조정, 선명도를 높이거나 낮춘다.
def sharpness(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.pillike.EnhanceSharpness()
        
        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

In [38]:
# 지정한 퍼센트만큼 이미지를 dropout 해준다.
def dropout(input_dir, output_dir):
    ia.seed(1)

    dir = input_dir
    images, annotations = read_train_dataset(dir)

    for idx in tqdm_notebook(range(len(images))):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Dropout(p=(0, 0.05)) # 이미지의 0퍼센트~5퍼센트를 랜덤으로 dropout
        
        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        new_image_file = output_dir + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(output_dir + annotations[idx][1])

1. original
2. noise
3. rot
4. blur
5. box
6. bright
8. noise_box
14. rot_bright

In [9]:
import os

In [10]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [11]:
def custom_aug(input_dir, output_dir):
    print("1. noise")
    noise_dir = os.path.join(output_dir, "noise/")
    createDirectory(noise_dir)
    noise(input_dir, noise_dir)
    
    print("2. rotation")
    rot_dir = os.path.join(output_dir, "rot/")
    createDirectory(rot_dir)
    rotation(input_dir, rot_dir)
    
    print("3. blur")
    blur_dir = os.path.join(output_dir, "blur/")
    createDirectory(blur_dir)
    blur(input_dir, blur_dir)
    
    print("4. box")
    box_dir = os.path.join(output_dir, "box/")
    createDirectory(box_dir)
    box(input_dir, box_dir)
    
    print("5. bright")
    bright_dir = os.path.join(output_dir, "bright/")
    createDirectory(bright_dir)
    bright(input_dir, bright_dir)
    
    print("6. noise_box")
    noise_box_dir = os.path.join(output_dir, "noise_box/")
    createDirectory(noise_box_dir)
    box(noise_dir, noise_box_dir)
    
    print("7. rot_bright")
    rot_bright_dir = os.path.join(output_dir, "rot_bright/")
    createDirectory(rot_bright_dir)
    bright(rot_dir, rot_bright_dir)

In [12]:
PROJECT_DIR = '/Users/kyh/Desktop/workspace'
temp = os.listdir(PROJECT_DIR)
file_list = []
for t in temp:
    if t != ".DS_Store" and t != "imgaug.ipynb" and t != ".ipynb_checkpoints":
        file_list.append(t)
print(file_list)

['18_pringles_onion', '17_changgu', '14_swingchip_hot', '15_turtle_chips', '20_pocachip_onion', '12_nacho_cheese', '13_oh_potato']


In [13]:
for file in tqdm_notebook(file_list):
    input_dir = os.path.join(PROJECT_DIR, os.path.join(file, "original/"))
    output_dir = os.path.join(PROJECT_DIR, file)
    print("Input : ", input_dir)
    print("Output : ", output_dir)
    custom_aug(input_dir, output_dir)

/Users/kyh/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/7 [00:00<?, ?it/s]

Input :  /Users/kyh/Desktop/workspace/18_pringles_onion/original/
Output :  /Users/kyh/Desktop/workspace/18_pringles_onion
1. noise


/Users/kyh/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1445 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

6. noise_box


  0%|          | 0/1000 [00:00<?, ?it/s]

7. rot_bright


  0%|          | 0/1000 [00:00<?, ?it/s]